# One run full walktrhough 

* Do the full walk through on the large data set
* Refactor the source code and bring it to individual scripts
* Ensure a full run with one click

# CRISP-DM

CRISP-DM stands for cross-industry process for data mining. The CRISP-DM methodology provides a structured approach to planning a data mining project.

![CRISP-DM.JPG](../reports/figures/CRISP-DM.jpg)

# Business Understanding

#### The scope of this project is to trace Corona Virus spread across the countries. 
* To trace the confirmed cases for all the countries
* To calculate the doubling rate.
* To simulate the spread of COVID-19 in Brazil using SIR model and also for 100+ countries.
* We would like to understand the data quality 
* Everything should be automated.
* To create a user friendly dashboard, which shows the current count of confirmed cases, doubling rate and SIR model.

In [1]:
## check some parameters
## depending where you launch your notebook, the relative path might not work
## you should start the notebook server from your base path
## when opening the notebook, typically your path will be ../ads_covid-19/notebooks
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: ads_covid-19'

## Update all data (Data Understanding)

In [4]:
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure. If there is no Repository 
        not present then clone the data from GitHub.
    '''
    
if os.path.exists('data/raw/COVID-19/'):
    print('Repository Exists: Fetch the latest data from repository')
    git_pull = subprocess.Popen( "git pull" ,
                         cwd = os.path.dirname('data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()
else:
    print('Repository not present. Fetch the entire repository')
    git_clone = subprocess.Popen( "git clone https://github.com/CSSEGISandData/COVID-19.git" ,
                         cwd = os.path.dirname('data/raw/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_clone.communicate()

if __name__ == '__main__':
    get_johns_hopkins()
   


Repository Exists: Fetch the latest data from repository


## Process pipeline (Data Preparation)

In [5]:
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_data():
    ''' Transformes the COVID data in a relational data set

    '''
    data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    time_idx = pd_raw.columns[4:]
    df_plot = pd.DataFrame({
        'date':time_idx})
    df_input_large= pd_raw['Country/Region'].unique()
    
    for each in df_input_large:
        df_plot[each] =np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))
    df = df_plot.drop('date', axis=1)
    
    #Merging the data set over COUNTRY for CODE column for worldmap
    df_code = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
    world_raw =  pd.DataFrame({"COUNTRY" : df_input_large, "Confirm cases" :df.iloc[-1]})
    world_con = pd.merge(world_raw, df_code, on = "COUNTRY").drop('GDP (BILLIONS)', axis=1)
    world_con.to_csv('data/processed/COVID_CRD.csv',sep=';',index=False)
    
    #Continuation of data preparation 
    pd_data_base=pd_raw.rename(columns={'Country/Region':'COUNTRY',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model_1=pd_data_base.set_index(['state','COUNTRY']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
    pd_relational_model = pd.merge(pd_relational_model_1, df_code, on = "COUNTRY").drop('GDP (BILLIONS)', axis=1)
    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
    
    pd_relational_model.to_csv('data/processed/20200823_COVID_relational_confirmed.csv',sep=';',index=False)
    
    
    #SIR model data preparation
    sir_plot = pd.DataFrame({
    'date':time_idx})
    #sir_plot.head()
    sir_arr= pd_raw['Country/Region'].unique()
    sir_list = sir_arr.tolist()
    for each in sir_list:
        sir_plot[each] =np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))
    #sir_plot.head()
    
    #Creating SIR plot for 100+ countries
    sir_plot= sir_plot.drop(columns = ['Taiwan*', 'South Sudan', 'Guyana','Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland',
                                   'Iraq', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
       'Korea, South', 'Kosovo','Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Luxembourg', 'MS Zaandam', 'Madagascar', 'Malawi',
       'Malaysia', 'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius',
       'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco',
       'Mozambique', 'Namibia', 'Nepal', 'Netherlands', 'New Zealand',
       'Nicaragua', 'Niger', 'Panama', 'Papua New Guinea', 'Paraguay',
       'Peru', 'Philippines', 'Bahamas', 'Egypt'])
    time_idx = [datetime.strptime(each, "%m/%d/%y") for each in sir_plot.date] #to convert all the dates into datetime 
    time_str= [each.strftime('%Y-%m-%d') for each in time_idx] #to convert datetime function to string
    #time_str[0:5]
    
    #Storing the processed data file and sep';' is a seperator [German std]
    sir_plot.to_csv('C:/Users/Vishal Sharbidar/Desktop/ADS_COVID-19/ads_covid-19/data/processed/COVID_sir_flat_table.csv', sep=';',index=False)
    
    
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.date)))
if __name__ == '__main__':

    store_relational_data()


 Number of rows stored: 59724
 Latest date is: 2020-09-14 00:00:00


## Filter and Doubling Rate Calculation (Modelling)

In [6]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','COUNTRY',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','COUNTRY']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter is a digital filter that can be applied to a set of digital data points for the purpose of 
        smoothing the data, that is, to increase the precision of the data without distorting the signal tendency.
        
        Parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression is used to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','COUNTRY',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','COUNTRY',filter_on]].groupby(['state','COUNTRY']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()



if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/20200823_COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['COUNTRY']=='Germany'].tail())


the test slope is: [2.]
            date state  COUNTRY  confirmed CODE  confirmed_filtered  \
32701 2020-09-10    no  Germany   258149.0  DEU            258018.2   
32702 2020-09-11    no  Germany   259735.0  DEU            259374.2   
32703 2020-09-12    no  Germany   260817.0  DEU            260732.0   
32704 2020-09-13    no  Germany   261737.0  DEU            261946.8   
32705 2020-09-14    no  Germany   263222.0  DEU            263161.6   

       confirmed_DR  confirmed_filtered_DR  
32701    160.722431             168.789051  
32702    156.332930             184.661656  
32703    194.577961             191.152480  
32704    260.502498             202.662158  
32705    217.817325             215.629569  


## SIR Modelling 

Susceptible Infected and Recovered model is used to fit the Gamma(Infection rate) and Beta(recovery rate) for each country by using the available data and I have also generated the fitted curve to show how well the processed data fits over truth. 

In [7]:
from scipy import optimize
from scipy import integrate

df_input_large=pd.read_csv('data/processed/COVID_sir_flat_table.csv',sep=';').iloc[80:]
pop = pd.read_csv('data/processed/population.csv',sep=';')

df_all = df_input_large.columns
df_all = list(df_all)

def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])


# Functions for SIR model with time step
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt


#Function defined for optimize curve fit
def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

#Fitting parameter for SIR model
for each in df_all[1:]:
    ydata = np.array(df_input_large[each])
    t=np.arange(len(ydata))
    N0 = 6000000 #max susceptible population

    # ensure re-initialization 
    I0=ydata[0]
    S0=N0-I0
    R0=0

    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata, maxfev = 1600)
    perr = np.sqrt(np.diag(pcov))

    # get the final fitted curve
    fitted=fit_odeint(t, *popt).reshape(-1,1)
    df_input_large[each +'_fitted'] = fitted 
    
df_input_large.to_csv('C:/Users/Vishal Sharbidar/Desktop/ADS_COVID-19/ads_covid-19/data/processed/COVID_sir_fitted_table.csv', sep=';',index=False)

C:\Users\Vishal Sharbidar\anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: overflow encountered in double_scalars
C:\Users\Vishal Sharbidar\anaconda3\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in double_scalars
C:\Users\Vishal Sharbidar\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\Users\Vishal Sharbidar\anaconda3\lib\site-packages\scipy\optimize\minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [8]:
df_input_large.head()

,date,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom_fitted,Uruguay_fitted,Uzbekistan_fitted,Venezuela_fitted,Vietnam_fitted,West Bank and Gaza_fitted,Western Sahara_fitted,Yemen_fitted,Zambia_fitted,Zimbabwe_fitted
80,4/11/20,555,433,1825,601,19,21,1975,967,6303,...,90273.000000,501.000000,767.000000,175.000000,258.000000,268.000000,4.000000,1.000000,40.000000,14.000000
81,4/12/20,607,446,1914,638,19,21,2142,1013,6315,...,92069.117881,505.128198,793.127761,183.367051,260.302021,277.397928,4.032049,1.074632,41.920408,14.730952
82,4/13/20,665,467,1983,646,19,23,2208,1039,6351,...,93893.461146,509.290409,820.137597,192.128973,262.624581,287.124098,4.064339,1.154834,43.932859,15.500027
83,4/14/20,714,475,2070,659,19,23,2277,1067,6415,...,95746.151377,513.486913,848.058732,201.303891,264.967863,297.189881,4.096869,1.241019,46.041748,16.309207
84,4/15/20,784,494,2160,673,19,23,2443,1111,6440,...,97627.295368,517.717993,876.921322,210.910716,267.332051,307.607034,4.129643,1.333634,48.251681,17.160578


## Visual Board

In [9]:
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State


import plotly.graph_objects as go
from plotly import tools

import os
print(os.getcwd())
df_input_large = pd.read_csv('data/processed/COVID_final_set.csv',sep=';')
df = pd.read_csv('data/processed/COVID_CRD.csv',sep=';')
df_input_sir = pd.read_csv('data/processed/COVID_sir_fitted_table.csv',sep=';')
df_all = df_input_sir.columns
df_all = list(df_all[:109])

'''Dashboard is created by using an external stylesheet named BOOTSTRAP. 
BOOTSTRAP allows us to divide the dashboard into Rows and columns.
COVID-19 dashbord has 5 Rows and 2 columns'''

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'COVID-19 Dashboard'

app.layout = html.Div([
        
        dbc.Row(dbc.Col(html.Div(dcc.Markdown('''
                            # Enterprise Data Science: COVID-19 Data Analytics
                            Goals of the project:
                            * To trace the confirmed cases for all the countries
                            * To calculate the doubling rate.
                            * To simulate the spread of COVID-19 in Brazil using SIR model and also for 100+ countries.
                            * To create a user friendly dashboard, which shows the current count of confirmed cases, doubling rate and SIR model.
                            ''')),
                        width={'size': 10, 'offset': 1},
                        )
                ),
        
        dbc.Row(
            [   #Dropdown for Timeline Confirmed and Doubling rate
                dbc.Col(dcc.Dropdown(
                            id='country_dropdown',
                            options=[ {'label': each,'value':each} for each in df_input_large['COUNTRY'].unique()],
                            value=['US', 'Germany','India'], # which are pre-selected
                            multi= True),
                        width={'size': 5, "offset": 0, 'order': 'first'}
                        ),
                #Dropdown for SIR model
                dbc.Col(dcc.Dropdown(
                            id='country_dropdown_sir',
                            options=[ {'label': each,'value':each} for each in df_all[1:]],
                            value='Brazil', # which are pre-selected
                            multi= False
                            ),
                        width={'size': 5, "offset": 2, 'order': 'second'}
                        ),
                ], no_gutters=True
        ),
    
        dbc.Row(
            [
                dbc.Col(
                        dcc.Dropdown(
                            id='doubling_time',
                            options=[
                                {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
                                {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
                                {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
                                {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'}
                            ],
                            value='confirmed',
                            multi=False
                            ),
                        width={'size': 3, "offset": 0, 'order': 'first'}
                        ),
                
                
                ], 
        ),
                
        dbc.Row(
            [
                dbc.Col(dcc.Graph( 
                            id='main_window_slope'
                            ),
                        width=6, md={'size': 5,  "offset": 0, 'order': 'first'}
                        ),
                
                dbc.Col(dcc.Graph(
                            id='SIR_model'
                            ),
                        width=6, md={'size': 5,  "offset": 1, 'order': 'last'}
                        ),
            ]
        ),
        
        dbc.Row(
                dbc.Col(dcc.Graph(id = "World_map",
                              figure = go.Figure(data = [go.Choropleth(
                                        locations = df['CODE'],
                                        z = df['Confirm cases'],
                                        text = df['COUNTRY'],
                                        colorscale = 'Blues',
                                        autocolorscale=False,
                                        reversescale=False,
                                        marker_line_color='darkgray',
                                        marker_line_width=0.5,
                                        colorbar_title = 'Confirmed cases'
                                        )],
                                        layout = go.Layout(
                                        title_text='COVID 19 WORLD MAP',
                                        height=1300,
                                        autosize = True,
                                        geo=dict(
                                            showframe=False,
                                            showcoastlines=False,
                                            projection_type='equirectangular'
                                        ))
                                     ),
                              
                              ),
                        width=12, md={'size': 12,  "offset": 0, 'order': 'first'}
                        ),
         )


])

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_dropdown', 'value'),
    Input('doubling_time', 'value')])

def update_figure(country_list,show_doubling):


    if 'DR' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['COUNTRY']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','COUNTRY','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['COUNTRY','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','COUNTRY','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['COUNTRY','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                height=900,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        ) 
    }

@app.callback(
    Output('SIR_model', 'figure'),
    [Input('country_dropdown_sir', 'value')])

def SIR_fig(con_input):
    df= df_input_sir
   
    
    for i in df[1:]:
        data = []
        trace = go.Scatter(x=df.date,
                        y=df[con_input],
                        mode='lines+markers',
                        name = con_input)
        data.append(trace)
        
        trace_fitted = go.Scatter(x=df.date,
                        y=df[con_input +'_fitted'], 
                        mode='lines+markers',
                        name=con_input+'_fitted')
        data.append(trace_fitted)
        
        
            
    return {'data': data,
            'layout' : dict(
                height=900,
                title= 'SIR model',
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },
                yaxis={'type':"log",
                       'range':'[1.1,5.5]'
                      }
                
            )
        }

if __name__ == '__main__':

    app.run_server(debug=True, port= 8051, use_reloader=False)


C:\Users\Vishal Sharbidar\Desktop\ADS_COVID-19\ads_covid-19
Dash is running on http://127.0.0.1:8051/

 in production, use a production WSGI server like gunicorn instead.



 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
